In [1]:
# import pandas
import pandas as pd

# load school data csv as dataframe
school_data_df = pd.read_csv("resources/schools_complete.csv")

# load student data csv as dataframe
student_data_df = pd.read_csv("resources/students_complete.csv")

# create list of suffixes & prefixes to replace
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

# replace suffixes and prefixes in student_data_df
for word in prefixes_suffixes:
    student_data_df["student_name"] = student_data_df["student_name"].str.replace(word,"",regex=True)

In [2]:
# import numpy
import numpy as np

In [3]:
# replace Thomas High School 9th grade reading scores with "NaN"
student_data_df.loc[(student_data_df["school_name"] == "Thomas High School") &
                    (student_data_df["grade"] == "9th"),
                   "reading_score"] = np.nan

In [4]:
# merge student_data_df and school_data_df
school_data_complete_df = pd.merge(student_data_df, school_data_df, on=["school_name", "school_name"])

In [5]:
# calculate total number of schools
school_count = len(school_data_complete_df["school_name"].unique())

# calculate total number of students
student_count = school_data_complete_df["Student ID"].count()

# calculate total budget
total_budget = school_data_df["budget"].sum()

# inspect total number of students
student_count

39170

In [6]:
# calculate average reading score
average_reading_score = school_data_complete_df["reading_score"].mean()

# calculate average math score
average_math_score = school_data_complete_df["math_score"].mean()

In [7]:
# calculate total number of Thomas High School 9th grade students
students_to_subtract = student_data_df.loc[(student_data_df["school_name"] == "Thomas High School") &
                                           (student_data_df["grade"] == "9th"),
                                           "Student ID"].count()

# inspect total number of Thomas High School 9th grade students
students_to_subtract

461

In [8]:
# adjust student count for purpose of calculating percentages
adjusted_student_count = student_count - students_to_subtract

# inspect adjusted student count
adjusted_student_count

38709

In [9]:
# calculate number of students with passing reading scores
passing_reading_count = school_data_complete_df.loc[(school_data_complete_df["reading_score"] >= 70),
                                                    "Student ID"].count()

# calculate number of students with passing math scores
passing_math_count = school_data_complete_df.loc[(school_data_complete_df["math_score"] >= 70),
                                                 "Student ID"].count()

# calculate number of students with passing reading scores and passing math scores
overall_passing_count = school_data_complete_df.loc[(school_data_complete_df["reading_score"] >= 70) &
                                                    (school_data_complete_df["math_score"] >= 70),
                                                    "Student ID"].count()

In [10]:
# calculate percentage of students with passing reading scores
passing_reading_percentage = passing_reading_count / adjusted_student_count * 100

# calculate percentage of students with passing math scores
passing_math_percentage = passing_math_count / adjusted_student_count * 100

# calculate percentage of students with passing reading scores and passing math scores
overall_passing_percentage = overall_passing_count / adjusted_student_count * 100

In [11]:
# create district summary dataframe
district_summary_df = pd.DataFrame(
    [{"Total Schools": school_count,
      "Total Students": student_count,
      "Total Budget": total_budget,
      "Average Math Score": average_math_score,
      "Average Reading Score": average_reading_score,
      "% Passing Math": passing_math_percentage,
      "% Passing Reading": passing_reading_percentage,
      "% Overall Passing": overall_passing_percentage}])

# format "Total Students" with comma for thousands separator
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)

# format the "Total Budget" with comma for a thousands separator, two decimal points, and dollar sign
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)

# format other columns with one decimal point
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.1f}".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.1f}".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.1f}".format)

# Display the data frame
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",79.0,81.9,75.9,85.7,64.9


In [12]:
# Determine the School Type
per_school_types = school_data_df.set_index(["school_name"])["type"]

# Calculate the total student count.
per_school_counts = school_data_complete_df["school_name"].value_counts()

# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete_df.groupby(["school_name"]).mean()["budget"]

# Calculate the per capita spending.
per_school_capita = per_school_budget / per_school_counts

# Calculate the average test scores.
per_school_math = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]
per_school_reading = school_data_complete_df.groupby(["school_name"]).mean()["reading_score"]

# Calculate the passing scores by creating a filtered DataFrame.
per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]
per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]

# Calculate the number of students passing math and passing reading by school.
per_school_passing_math = per_school_passing_math.groupby(["school_name"]).count()["student_name"]
per_school_passing_reading = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]

# Calculate the percentage of passing math and reading scores per school.
per_school_passing_math = per_school_passing_math / per_school_counts * 100
per_school_passing_reading = per_school_passing_reading / per_school_counts * 100

# Calculate the students who passed both reading and math.
per_passing_math_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)
                                                   & (school_data_complete_df["math_score"] >= 70)]

# Calculate the number of students passing math and passing reading by school.
per_passing_math_reading = per_passing_math_reading.groupby(["school_name"]).count()["student_name"]

# Calculate the percentage of passing math and reading scores per school.
per_overall_passing_percentage = per_passing_math_reading / per_school_counts * 100

In [13]:
# Create the DataFrame
per_school_summary_df = pd.DataFrame({
    "School Type": per_school_types,
    "Total Students": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_school_capita,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    "% Passing Math": per_school_passing_math,
    "% Passing Reading": per_school_passing_reading,
    "% Overall Passing": per_overall_passing_percentage})

# Format the Total School Budget and the Per Student Budget
per_school_summary_df["Total School Budget"] = per_school_summary_df["Total School Budget"].map("${:,.2f}".format)
per_school_summary_df["Per Student Budget"] = per_school_summary_df["Per Student Budget"].map("${:,.2f}".format)

In [14]:
# Get the number of 10th-12th graders from Thomas High School (THS)
thomas_10th_thru_12th = student_data_df.loc[(student_data_df["school_name"] == "Thomas High School") &
                                           (student_data_df["grade"] != "9th"),
                                           "Student ID"].count()

In [15]:
passing_math_Thomas = student_data_df.loc[
    (student_data_df["school_name"] == "Thomas High School") &
    (student_data_df["grade"] != "9th") &
    (student_data_df["math_score"] >= 70),
    "Student ID"].count()

passing_reading_Thomas = student_data_df.loc[
    (student_data_df["school_name"] == "Thomas High School") &
    (student_data_df["grade"] != "9th") &
    (student_data_df["reading_score"] >= 70),
    "Student ID"].count()

overall_passing_Thomas = student_data_df.loc[
    (student_data_df["school_name"] == "Thomas High School") &
    (student_data_df["grade"] != "9th") &
    (student_data_df["reading_score"] >= 70) &
    (student_data_df["math_score"] >= 70),
    "Student ID"].count()

In [16]:
# calculate percentage of students with passing math scores
Thomas_math_percentage = passing_math_Thomas / thomas_10th_thru_12th * 100

# calculate percentage of students with passing reading scores
Thomas_reading_percentage = passing_reading_Thomas / thomas_10th_thru_12th * 100

# calculate percentage of students with passing reading scores and passing math scores
Thomas_overall_percentage = overall_passing_Thomas / thomas_10th_thru_12th * 100

In [17]:
# Replace the passing math percent for Thomas High School in the per_school_summary_df.
per_school_summary_df.loc["Thomas High School",
                          "% Passing Math"] = Thomas_math_percentage

In [18]:
# Replace the passing reading percent for Thomas High School in the per_school_summary_df.
per_school_summary_df.loc["Thomas High School",
                          "% Passing Reading"] = Thomas_reading_percentage

In [19]:
# Step 12. Replace the passing overall percent for Thomas High School in the per_school_summary_df.
per_school_summary_df.loc["Thomas High School",
                          "% Overall Passing"] = Thomas_overall_percentage

In [20]:
per_school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [21]:
# Sort and show top five schools.
top_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=False)

top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.896082,93.185690,97.018739,90.630324
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [22]:
# Sort and show bottom five schools.
bottom_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=True)

bottom_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [23]:
# Create a grade level DataFrames.
ninth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "9th")]

tenth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "10th")]

eleventh_graders = school_data_complete_df[(school_data_complete_df["grade"] == "11th")]

twelfth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "12th")]

In [24]:
# Group each grade level DataFrame by the school name for the average math score.
ninth_grade_math_scores = ninth_graders.groupby(["school_name"]).mean()["math_score"]

tenth_grade_math_scores = tenth_graders.groupby(["school_name"]).mean()["math_score"]

eleventh_grade_math_scores = eleventh_graders.groupby(["school_name"]).mean()["math_score"]

twelfth_grade_math_scores = twelfth_graders.groupby(["school_name"]).mean()["math_score"]

In [25]:
# Group each grade level DataFrame by the school name for the average reading score.
ninth_grade_reading_scores = ninth_graders.groupby(["school_name"]).mean()["reading_score"]

tenth_grade_reading_scores = tenth_graders.groupby(["school_name"]).mean()["reading_score"]

eleventh_grade_reading_scores = eleventh_graders.groupby(["school_name"]).mean()["reading_score"]

twelfth_grade_reading_scores = twelfth_graders.groupby(["school_name"]).mean()["reading_score"]

In [26]:
# Combine each grade level Series for average math scores by school into a single DataFrame.
math_scores_by_grade = pd.DataFrame({
               "9th": ninth_grade_math_scores,
               "10th": tenth_grade_math_scores,
               "11th": eleventh_grade_math_scores,
               "12th": twelfth_grade_math_scores})

math_scores_by_grade["9th"] = math_scores_by_grade["9th"].map("{:,.1f}".format)
math_scores_by_grade["10th"] = math_scores_by_grade["10th"].map("{:,.1f}".format)
math_scores_by_grade["11th"] = math_scores_by_grade["11th"].map("{:,.1f}".format)
math_scores_by_grade["12th"] = math_scores_by_grade["12th"].map("{:,.1f}".format)

math_scores_by_grade.index.name = None

math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9
Pena High School,83.6,83.4,84.3,84.1


In [27]:
# Combine each grade level Series for average reading scores by school into a single DataFrame.
reading_scores_by_grade = pd.DataFrame({
              "9th": ninth_grade_reading_scores,
              "10th": tenth_grade_reading_scores,
              "11th": eleventh_grade_reading_scores,
              "12th": twelfth_grade_reading_scores})

reading_scores_by_grade["9th"] = reading_scores_by_grade["9th"].map("{:,.1f}".format)
reading_scores_by_grade["10th"] = reading_scores_by_grade["10th"].map("{:,.1f}".format)
reading_scores_by_grade["11th"] = reading_scores_by_grade["11th"].map("{:,.1f}".format)
reading_scores_by_grade["12th"] = reading_scores_by_grade["12th"].map("{:,.1f}".format)

reading_scores_by_grade.index.name = None

reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2
Pena High School,83.8,83.6,84.3,84.6


In [28]:
# Cut the per_school_capita into the spending ranges.
spending_bins = [0, 585, 630, 645, 675]
per_school_capita.groupby(pd.cut(per_school_capita, spending_bins)).count()

(0, 585]      4
(585, 630]    4
(630, 645]    4
(645, 675]    3
dtype: int64

In [29]:
# Establish the spending bins and group names.
spending_bins = [0, 585, 630, 645, 675]
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]

In [30]:
# Categorize spending based on the bins.
per_school_summary_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=group_names)

In [31]:
# Calculate averages for the desired columns.
spending_math_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]

In [32]:
spending_summary_df = pd.DataFrame({
          "Average Math Score" : spending_math_scores,
          "Average Reading Score": spending_reading_scores})
spending_summary_df["Average Math Score"] = spending_summary_df["Average Math Score"].map("{:.1f}".format)
spending_summary_df["Average Reading Score"] = spending_summary_df["Average Reading Score"].map("{:.1f}".format)
spending_summary_df

,Average Math Score,Average Reading Score
Spending Ranges (Per Student),,
<$584,83.5,83.9
$585-629,81.9,83.2
$630-644,78.5,81.6
$645-675,77.0,81.0


In [33]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [34]:
# Categorize spending based on the bins.
per_school_summary_df["School Size"] = pd.cut(per_school_summary_df["Total Students"], size_bins, labels=group_names)

In [35]:
# Calculate averages for the desired columns.
size_math_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]

In [36]:
# Assemble into DataFrame.
# Assemble into DataFrame.
size_summary_df = pd.DataFrame({
          "Average Math Score" : size_math_scores,
          "Average Reading Score": size_reading_scores})
size_summary_df["Average Math Score"] = size_summary_df["Average Math Score"].map("{:.1f}".format)
size_summary_df["Average Reading Score"] = size_summary_df["Average Reading Score"].map("{:.1f}".format)
size_summary_df

,Average Math Score,Average Reading Score
School Size,,
Small (<1000),83.8,83.9
Medium (1000-2000),83.4,83.9
Large (2000-5000),77.7,81.3


In [37]:
# Calculate averages for the desired columns.
type_math_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Math Score"]

type_reading_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Reading Score"]

type_summary_df = pd.DataFrame({
          "Average Math Score" : type_math_scores,
          "Average Reading Score": type_reading_scores})

type_summary_df["Average Math Score"] = type_summary_df["Average Math Score"].map("{:.1f}".format)

type_summary_df["Average Reading Score"] = type_summary_df["Average Reading Score"].map("{:.1f}".format)

type_summary_df

,Average Math Score,Average Reading Score
School Type,,
Charter,83.5,83.9
District,77.0,81.0
